In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# Read the data and seperate by ;
data = pd.read_csv('gdp_data.csv', sep=';')

data.drop(data.columns[0], axis=1, inplace=True)

data

,income,capital,labor
0,114043,8310,182113
1,120410,8529,193749
2,129187,8738,205192
3,134705,8952,215130
4,139960,9171,225021
5,150511,9569,237026
6,157897,9527,248897
7,165286,9662,260661
8,178491,10334,275466
9,199457,10981,295378


In [9]:
data['ln_capital'] = np.log(data['capital'])
data['ln_labor'] = np.log(data['labor'])

data

,income,capital,labor,ln_capital,ln_labor
0,114043,8310,182113,9.025215,12.112383
1,120410,8529,193749,9.051227,12.174319
2,129187,8738,205192,9.075437,12.231701
3,134705,8952,215130,9.099632,12.278998
4,139960,9171,225021,9.123802,12.323949
5,150511,9569,237026,9.166284,12.375925
6,157897,9527,248897,9.161885,12.424794
7,165286,9662,260661,9.175956,12.470976
8,178491,10334,275466,9.243195,12.526219
9,199457,10981,295378,9.303922,12.596011


In [12]:
# Add constant for the intercept term
X = sm.add_constant(data[['ln_capital', 'ln_labor']])

# Fit the linear regression model
model = sm.OLS(data['income'], X).fit()

# Print model summary
print(model.summary())

# Print coefficients
print('Coefficients:', model.params)

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     971.4
Date:                Mon, 08 Apr 2024   Prob (F-statistic):           2.99e-18
Time:                        13:25:09   Log-Likelihood:                -206.83
No. Observations:                  20   AIC:                             419.7
Df Residuals:                      17   BIC:                             422.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.518e+06   1.76e+05    -14.271      0.0

Overall, based on the high R-squared value, low p-value of the F-statistic, and significant coefficient of ln_labor, we can conclude that the model provides an excellent fit to the data in explaining the variation in income. However, the non-significant coefficient for ln_capital suggests that this variable might not be a useful predictor of income in this model. Additionally, the low Durbin-Watson statistic indicates potential issues with autocorrelation in the residuals, which might need further investigation or model refinement.

c)

In [20]:
def normal_distribution(n):
    return np.random.normal(0, 1, n)

psudo_random_data = normal_distribution(20)

array([ 0.09149784, -0.69611556,  0.59516251,  0.83205409, -1.23584281,
        0.37859117,  0.59994808, -1.59798856,  0.05373762,  1.66015184,
        0.16576955, -0.2962754 ,  0.03540175,  0.52093703, -0.33390525,
        0.7636647 , -0.96223464,  0.81933144,  1.40628161, -0.51917907])